This notebook contains the code which applies the Lipinski rule to our data


In [ ]:
!pip install rdkit-pypi
!pip install scikit-learn
!pip install git+https://github.com/bp-kelley/descriptastorus


In [ ]:
from collections import defaultdict
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from descriptastorus.descriptors import rdDescriptors, rdNormalizedDescriptors
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
best = pd.read_csv('new-sub-tuned-2.csv')

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw

from rdkit.Chem import PandasTools, Fragments, Descriptors, rdMolDescriptors, Graphs
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem.FilterCatalog import FilterCatalog, FilterCatalogParams
from rdkit.Chem.Draw import IPythonConsole

rdMolDescriptors.NumRotatableBondsOptions = 'NonStrict'

def calc_molecular_properties(smiles):
  mol  = Chem.MolFromSmiles(smiles)
  mw   = rdMolDescriptors.CalcExactMolWt(mol)
  hba  = rdMolDescriptors.CalcNumLipinskiHBA(mol)
  hbd  = rdMolDescriptors.CalcNumLipinskiHBD(mol)
  
  logp = rdMolDescriptors.CalcCrippenDescriptors(mol)[0]
  mr   = rdMolDescriptors.CalcCrippenDescriptors(mol)[1]
  tpsa = rdMolDescriptors.CalcTPSA(mol)
  return pd.Series(
        [mw, hba, hbd, logp, mr, tpsa],
        index=["molecular_weight", "hba", "hbd", "logp", "refract", "TPSA"])

In [ ]:
best['mol_weight'] = best.Smiles.apply(lambda x: calc_molecular_properties(x)['molecular_weight'])
best['refract'] = best.Smiles.apply(lambda x: calc_molecular_properties(x)['refract'])

In [ ]:
best[(best['mol_weight'] < 180) | (best['refract'] < 40) & (best['Active'] == 1)]

In [1]:
best.loc[(best['mol_weight'] < 180) & (best['refract'] < 40), 'Active'] = 0
best = best[['Smiles', 'Active']]
best.Active = best.Active.apply(lambda x: int(x))

NameError: name 'best' is not defined

In [ ]:
best.to_csv('new_best_best_best.csv', index=False, sep=',')